In [1]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, Span
from bokeh.io import curdoc
from bokeh.themes import built_in_themes, Theme

from bokeh.layouts import row
import time
import pytz
import zoneinfo
from pytz import timezone

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [117]:
base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)

#clean df
colNames=[i[len('health_description_'):] for i in list(df_health_description.columns)]
col_names_dict={i:j for i,j in zip(list(df_health_description.columns),colNames)}
df_health_description.rename(columns=col_names_dict, inplace=True)
df_health_description=df_health_description.sort_values(by=['datetime_of_activity'], ascending=False)

In [3]:
def plot_text_format(x):
    return ('%.1f' % x).rstrip('0').rstrip('.')

In [118]:
#assign default chart dates
date_end=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[0],'%Y-%m-%d %H:%M:%S.%f')
date_end=date_end+ timedelta(days=1)
date_start=date_end- timedelta(days=7)

#get cardio performance metric into lists
df1=df_health_description.loc[df_health_description.metric1_carido<100]#filter dataset
obs_x1=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df1.datetime_of_activity]
obs_y1=df1.metric1_carido
obs_y1_formatted=[ plot_text_format(i) for i in obs_y1]

#get activities into lists
df2=df_health_description.loc[(df_health_description.var_type=='Activity')]#filter dataset
obs_x2=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df2.datetime_of_activity]
obs_y2=df2.var_activity.to_list()

#get weights into lists
df3=df_health_description.loc[(df_health_description.var_type=='Weight')]#filter dataset
obs_x3=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df3.datetime_of_activity]
obs_y3=df3.metric3.to_list()
obs_y3_min=min(obs_y3)
obs_y3_adjusted=[i-obs_y3_min+10 for i in obs_y3]

#create figure object [start of jupyter notebook]
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date_start,date_end),y_range=(-10,90),width=900, height=400)


ValueError: min() arg is an empty sequence

In [46]:
# obs_y3_min=obs_y3.min()
obs_y3_adjusted

[13.0, 12.0, 10.0]

In [47]:
#add cardio_metric1
if len(obs_x1)>0:
    circle=fig1.circle(obs_x1,obs_y1, legend_label="Cardio Performance", fill_color='#c77711', line_color=None,
                  size=20)
    source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1_formatted))
    glyph1 = Text(text="text",text_font_size={'value': '10px'},x_offset=-10, y_offset=5)
    fig1.add_glyph(source1, glyph1)

#add activities to fig1
if len(obs_x2)>0:
    for a,b in zip(obs_x2,obs_y2):
        #add activity data
        source2 = ColumnDataSource(dict(x=[a], y=[80], text=[b]))
        glyph2 = Text(text="text", text_color="#414444", text_font_size={'value': '10px'},
                     x_offset=-10,angle=-1.58)

        #add line for activity data
        line_start_time=time.mktime(a.timetuple())*1000
        important_time = Span(location=line_start_time, dimension='height', line_color="#414444", line_dash='dashed', line_width=1)
        fig1.add_glyph(source2, glyph2)
        fig1.add_layout(important_time)

    fig1.add_glyph(source2, glyph2)

In [48]:
#add weight as triangles
if len(obs_x3)>0:
    triangle=fig1.triangle(obs_x3,obs_y3_adjusted, legend_label="Weight", fill_color='#c77711', line_color=None,
                  size=20)
    source3 = ColumnDataSource(dict(x=obs_x3, y=obs_y3_adjusted, text=obs_y3))
    glyph3 = Text(text="text",text_font_size={'value': '10px'},x_offset=-10, y_offset=5)
    fig1.add_glyph(source3, glyph3)

In [49]:
fig1.ygrid.grid_line_color = None
fig1.yaxis.major_label_text_color = None
fig1.yaxis.major_tick_line_color = None
fig1.yaxis.minor_tick_line_color = None

fig1.legend.background_fill_color = "#578582"
fig1.legend.background_fill_alpha = 0.2
fig1.legend.border_line_color = None

theme_1=curdoc().theme = Theme(filename=proxyApp.config['BOKEH_THEME'])
curdoc().theme = theme_1
show(fig1)

#scratech work delte record

In [51]:
delete_record_id=252
db.session.query(Health_description).filter(Health_description.id==delete_record_id).delete()
db.session.commit()

In [131]:
# date_list=obs_x2
type(df_health_description.datetime_of_activity.max())

str

In [107]:
date_end=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[-1],'%Y-%m-%d %H:%M:%S.%f')
date_end=date_end+ timedelta(days=1)

date_end

datetime.datetime(2021, 4, 26, 12, 17, 19, 88000)

In [108]:
def max_date(date_end,date_list):
    new_max=max(date_list)
    if new_max>date_end:
        date_end=new_max
    return date_end

In [109]:
date_end=max_date(date_end,obs_x2)
date_end

datetime.datetime(2021, 8, 20, 8, 19)

In [102]:
obs_x4=[datetime.datetime(2021,7,6),datetime.datetime(2021,7,9)]

In [116]:
try:
    date_end=max_date(date_end,obs_x5)
except:
    pass
    
date_end
    

datetime.datetime(2021, 8, 20, 8, 19)

In [72]:
try:
    date_list=date_list+obs_x4
except:
    pass

In [111]:
obs_x5

NameError: name 'obs_x5' is not defined

In [114]:
#scenario#1: Add empty list to a list without altering it
date_end

datetime.datetime(2021, 8, 20, 8, 19)